This is an example of a RAG search engine powered by Openai and Pinecone

In [27]:
# Import necessary libraries
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Load environment variables
load_dotenv()

# Set up Pinecone API key and initialize
pinecone_api_key = os.getenv("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY is not set in the .env file")

pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI()

# Connect to the index
index_name = 'idea-index'
namespace = os.getenv("PINECONE_NAMESPACE")
index = pc.Index(index_name)

# Define functions
def get_embeddings(text, model="text-embedding-3-large"):
    """Generate embeddings using OpenAI."""
    response = client.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding

def get_chat_completion(messages, model, response_format=None):
    """Generate chat completion using OpenAI."""
    completion_params = {
        "model": model,
        "messages": messages
    }
    
    if response_format:
        completion_params["response_format"] = response_format

    completion = client.chat.completions.create(**completion_params)

    result = completion.choices[0].message.content

    return result

def build_cosine_query(query_text):
    """Build a cosine similarity query."""

    messages = [
        {"role": "developer", "content": "You are a helpful innovation assistant. Format my question in a way that would get the best results from a vector search of ideas in my database. The ideas have been embedded with their Title, Description, and Comments combined and lemmatized before uploading. I am going to take the output of this and give it to Pinecone to perform a cosine similarity search."},
        {"role": "user", "content": "I want to find ideas related to renewable energy sources."},
        {"role": "assistant", "content": "Looking for innovative ideas, projects, or discussions related to renewable energy sources (e.g. solar, wind, hydroelectric, geothermal, biomass), clean energy technologies, sustainable power generation, reduced carbon footprint, zero-emission strategies, and other eco-friendly solutions."},
        {"role": "user", "content": "What are some innovative ideas for increasing revenue?"},
        {"role": "assistant", "content": "Seeking creative and novel concepts, strategies, or initiatives to boost income, generate profits, enhance financial performance, drive sales growth, improve monetization, optimize revenue streams, or maximize earnings."},
        {"role": "user", "content": "How can I improve customer satisfaction in my business?"},
        {"role": "assistant", "content": "Exploring methods, techniques, approaches, or practices to enhance customer experience, increase client happiness, elevate service quality, improve customer relations, foster client loyalty, or deliver exceptional customer service."},
        {"role": "user", "content": "What are some ideas for reducing waste in manufacturing processes?"},
        {"role": "assistant", "content": "Investigating innovative solutions, strategies, technologies, or methodologies to minimize waste production, reduce resource consumption, optimize material efficiency, enhance sustainability, or implement eco-friendly practices in manufacturing operations."},
        {"role": "user", "content": query_text}
    ]

    response_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "query_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "result": {
                        "description": "The reformatted query string to use for cosine similarity search on the vector database.",
                        "type": "string"
                    }
                },
                "additionalProperties": False
            }
        }
    }

    response = get_chat_completion(messages, model="gpt-4o", response_format=response_format)

    # Parse the JSON response
    response_json = json.loads(response)

    query = response_json["result"]  # Safely access "result" key from JSON response

    return query

def query_pinecone(query_text, index, top_k=10):
    """Query Pinecone index with a text input."""
    query_embedding = get_embeddings(query_text)
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace
    )

    # Convert response to a dictionary
    response_dict = response.to_dict()  # Use Pinecone's `to_dict()` method if available

    return response_dict

def generate_response(query_text, response_dict):

    messages = [
        {"role": "developer", 
         "content": (
            "You are a helpful innovation assistant. Use the following ideas to answer the user's question."
            f"Use the following ideas to answer the user's question:\n"
            f"{json.dumps(response_dict, indent=2)}\n"  # Serialize response to JSON format
            f"### Instructions:\n"
            f"- Focus your response on the user's query.\n"
            f"- Use the provided ideas to give specific, actionable answers.\n"
            f"- If no ideas seem directly relevant, provide general advice or suggest next steps for the user to refine their question.\n\n"
            f"- Summarize the key points from the ideas that are most relevant to the user's query as one narrative.\n"
            f"- Provide a clear and concise response that addresses the user's query based on the ideas provided. Do not just list the ideas."
        )},
        {"role": "user", "content": query_text}
    ]

    respone_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "response_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "response": {
                        "description": "The response to the user's query based on the retrieved ideas.",
                        "type": "string"
                    }
                },
                "additionalProperties": False
            }
        }
    }

    response = get_chat_completion(messages, model="gpt-4o", response_format=respone_format)

    # Parse the JSON response
    response_json = json.loads(response)

    query = response_json["response"]  # Safely access "result" key from JSON response
    
    return query

# Interactive Cells
# Query Example
query_text = input("\nEnter a query to search the ideaspace: ")

cosine_query = build_cosine_query(query_text)
print(f"\nCosine Query: {cosine_query}\n")

pinecone_results = query_pinecone(cosine_query, index)

response = generate_response(query_text, pinecone_results)

print(response)



Cosine Query: Searching for concepts, designs, or implementations related to customizable dashboards, real-time data visualization, interactive user interfaces, personalized analytics displays, dynamic information panels, and adaptable reporting platforms.

There are several ideas related to creating configurable dashboards that cater to individual or organizational needs. One prominent concept is the development of a tool that enables users to build their own analytics dashboards without needing technical assistance. This tool would allow users to use a drag-and-drop interface similar to Excel Pivot Tables or Salesforce Report Builder to customize dashboards by selecting specific data fields and visual formats, producing graphical charts tailored to their needs. Access to such a feature could potentially be monetized by offering different tiers, such as 'Analytics Simple' versus 'Analytics Pro'.

Another suggestion is to enhance dashboards by adding widgets that provide quick access 